In [1]:
import numpy as np
import scipy as sp
import pandas as pd
import mne
from hyperframe.frame import DataFrame
from mne.io import read_raw_ant
import os
from matplotlib.lines import lineStyles
from scipy.signal import butter, sosfiltfilt
from scipy.stats import zscore
from scipy.ndimage import label
from combined_analysis_bachelor.code.functions_for_pipeline import get_ch_indices, plot_channel_overview, normalize_emg, \
    notched_and_filtered, create_df, envelope, rectify, tkeo
import matplotlib
matplotlib.use('Qt5Agg')
import matplotlib.pyplot as plt

channel_custom_order = ["BIP3", "BIP4", "BIP5", "BIP9", "BIP10", "BIP12", "BIP6", "BIP1", "BIP2", "BIP11", "BIP8", "BIP7"]
EMG = ["BIP7", "BIP8", "BIP9", "BIP10", "BIP11", "BIP12"]
ACC = ["BIP1", "BIP2", "BIP3", "BIP4", "BIP5", "BIP6"]
locations = {"BIP7":"right M. tibialis anterior",
            "BIP8": "right M. deltoideus",
            "BIP11": "right M. brachioradialis",
            "BIP1" : "ACC right hand : y",
            "BIP2" : "ACC right hand : z",
            "BIP6" : "ACC right hand : x",
            "BIP3" : "ACC left hand : x",
            "BIP4" : "ACC left hand : y",
            "BIP5" : "ACC left hand : z",
            "BIP9" : "left M. brachioradialis",
            "BIP10" : "left M. deltoideus",
            "BIP12" : "left M. tibialis anterior"}
sf=1000

In [25]:
# get left forearm:
A_move1_filtered = pd.read_hdf("C:/Users/User/Documents/bachelorarbeit/data/EMG_ACC/sub-91/"
                              "PTB_01_data_processed/A_1.2_move_processed.h5", key="data")
A_move1_filtered_df = pd.DataFrame(A_move1_filtered)


### location: left forearm ###
left_forearm_move = A_move1_filtered_df["BIP9"]
left_forearm_move *= 1e6

# rectifying and building envelope #
rectified_move = left_forearm_move.abs()
low_pass = 4/(1000/2)
sos = butter(4, low_pass, btype='lowpass', output="sos")
enveloped_move1 = sosfiltfilt(sos, x=rectified_move)

In [26]:
# taking baseline section
A_move_1_baseline = enveloped_move1[7*sf:17*sf] # 10 secs for baseline values
p99 = np.percentile(A_move_1_baseline, 99)
mu_move1_baseline, sigma_move_baseline = A_move_1_baseline.mean(), A_move_1_baseline.std()

In [27]:
# set thresh and get activity
p99_thresh = p99 * 4
std_thresh1 = mu_move1_baseline + 12*sigma_move_baseline
envelope_activity1 = enveloped_move1 > std_thresh1

In [28]:
# get on and offsets
min_samples = int(1 * sf)         # 1000-ms-Grenze

labels, n_lbl = label(envelope_activity1)
valid = np.zeros_like(envelope_activity1, dtype=bool)

for lbl in range(1, n_lbl + 1):
    idx = np.where(labels == lbl)[0]
    if idx.size >= min_samples:
        valid[idx] = True

# On- & Offsets
Onsets  = np.where(np.diff(valid.astype(int)) ==  1)[0] + 1
Offsets = np.where(np.diff(valid.astype(int)) == -1)[0] + 1

In [29]:
plt.plot(A_move1_filtered_df["Sync_Time (s)"], enveloped_move1, "b", label="Envelope")
#plt.plot(A_move1_filtered_df["Sync_Time (s)"][:-1], np.diff(enveloped_move), "green", label="diff envelope")
plt.axhline(std_thresh1, color="r", linestyle="--", label="Threshold")
#plt.axhline(diff_p99_thresh, color="r", linestyle="-.", label="thresh for diff")
for x in Onsets:
    plt.axvline(A_move1_filtered_df["Sync_Time (s)"][x], ls="--", c="g")
for x in Offsets:
    plt.axvline(A_move1_filtered_df["Sync_Time (s)"][x], ls="--", c="k")
#for y in diff_onsets:
 #   plt.axvline(A_move1_filtered_df["Sync_Time (s)"][y], ls="-.", c="g")
#for y in diff_offsets:
#    plt.axvline(A_move1_filtered_df["Sync_Time (s)"][y], ls="-.", c="k")
plt.legend()
plt.xlabel("Time (s)")
plt.ylabel("Amplitude (µV)")
plt.title("Muscle Activity Detection for move 1 -> left forearm - thresh: mean+12*std")
#plt.xlim(43,47)
plt.show()

In [103]:
### NOW FOR SAME ARM BUT SECOND MOVE ###

In [76]:
A_move2_filtered = pd.read_hdf("C:/Users/User/Documents/bachelorarbeit/data/EMG_ACC/sub-91/"
                              "PTB_01_data_processed/A_1.5_move_processed.h5", key="data")
A_move2_filtered_df = pd.DataFrame(A_move2_filtered)


### location: left forearm ###
left_forearm_move = A_move2_filtered_df["BIP9"]
left_forearm_move *= 1e6

# rectifying and building envelope #
rectified_move = left_forearm_move.abs()
low_pass = 4/(1000/2)
sos = butter(4, low_pass, btype='lowpass', output="sos")
enveloped_move2 = sosfiltfilt(sos, x=rectified_move)

In [3]:
# taking baseline section
A_move_2_baseline = enveloped_move2[9*sf:15*sf] # 10 secs for baseline values
p99 = np.percentile(A_move_2_baseline, 99)
mu_move2_baseline, sigma_move2_baseline = A_move_2_baseline.mean(), A_move_2_baseline.std()

In [4]:
# set thresh and get activity
p99_thresh = p99 * 4
std_thresh_move2 = mu_move2_baseline + 12 * sigma_move2_baseline
envelope_activity2 = enveloped_move2 > std_thresh_move2

In [10]:
# get on and offsets
min_samples = int(1 * sf)         # 1000-ms-Grenze

labels, n_lbl = label(envelope_activity2)
valid = np.zeros_like(envelope_activity2, dtype=bool)

for lbl in range(1, n_lbl + 1):
    idx = np.where(labels == lbl)[0]
    if idx.size >= min_samples:
        valid[idx] = True

# On- & Offsets
Onsets_move2  = np.where(np.diff(valid.astype(int)) ==  1)[0] + 1
Offsets_move2 = np.where(np.diff(valid.astype(int)) == -1)[0] + 1
print(Onsets_move2)
print(Offsets_move2)

[16119 20885 25598 31021 36088 41097 45979 51101 52273 56057 61157 65916
 71154 75862 80749 86037]
[18623 23482 28500 33604 38728 43759 48680 52217 53863 58642 63895 68798
 74224 78607 83520 89005]


In [ ]:
plt.plot(A_move2_filtered_df["Sync_Time (s)"], enveloped_move2, "b", label="Envelope")
#plt.plot(A_move1_filtered_df["Sync_Time (s)"][:-1], np.diff(enveloped_move), "green", label="diff envelope")
plt.axhline(std_thresh_move2, color="r", linestyle="--", label="Threshold")
#plt.axhline(diff_p99_thresh, color="r", linestyle="-.", label="thresh for diff")
for x in Onsets_move2:
    plt.axvline(A_move1_filtered_df["Sync_Time (s)"][x], ls="--", c="g")
for x in Offsets_move2:
    plt.axvline(A_move1_filtered_df["Sync_Time (s)"][x], ls="--", c="k")
#for y in diff_onsets:
 #   plt.axvline(A_move1_filtered_df["Sync_Time (s)"][y], ls="-.", c="g")
#for y in diff_offsets:
#    plt.axvline(A_move1_filtered_df["Sync_Time (s)"][y], ls="-.", c="k")
plt.legend()
plt.xlabel("Time (s)")
plt.ylabel("Amplitude (µV)")
plt.title("Muscle Activity Detection for move 2 -> left forearm - thresh: mean+12*std")
#plt.xlim(43,47)
plt.show()

In [16]:
## removing off and onsets that are too close together (splitting one arm raise)
def take_out_short_off_onset(onsets, offsets, min_time_period, sampling_frequency):
    onsets_clean = onsets.copy()
    offsets_clean = offsets.copy()
    min_sample_period = min_time_period * sampling_frequency
    
    if hasattr(onsets, 'tolist'):
        onsets_clean = onsets.tolist()
        offsets_clean = offsets.tolist()
    
    # Iterate in reverse to safely delete items
    for i in range(len(offsets) - 2, -1, -1):  # Start from the end
        time_between = onsets[i+1] - offsets[i]
        if time_between <= min_sample_period:
            del onsets_clean[i + 1]
            del offsets_clean[i]
    
    return onsets_clean, offsets_clean

In [20]:
new_onsets_move2, new_offsets_move2 = take_out_short_off_onset(Onsets_move2, Offsets_move2, 1, sf)

In [22]:
print(new_onsets_move2)
print(new_offsets_move2)

[16119, 20885, 25598, 31021, 36088, 41097, 45979, 51101, 56057, 61157, 65916, 71154, 75862, 80749, 86037]
[18623, 23482, 28500, 33604, 38728, 43759, 48680, 53863, 58642, 63895, 68798, 74224, 78607, 83520, 89005]


In [24]:
plt.plot(A_move2_filtered_df["Sync_Time (s)"], enveloped_move2, "b", label="Envelope")
#plt.plot(A_move1_filtered_df["Sync_Time (s)"][:-1], np.diff(enveloped_move), "green", label="diff envelope")
plt.axhline(std_thresh_move2, color="r", linestyle="--", label="Threshold")
for x in new_onsets_move2:
    plt.axvline(A_move2_filtered_df["Sync_Time (s)"][x], ls="--", c="g")
for x in new_offsets_move2:
    plt.axvline(A_move2_filtered_df["Sync_Time (s)"][x], ls="--", c="k")
plt.legend()
plt.xlabel("Time (s)")
plt.ylabel("Amplitude (µV)")
plt.title("Muscle Activity Detection for move 2 -> left forearm - thresh: mean+12*std")
plt.show()

In [30]:
# ====== now with Delt L, move2 ================

In [77]:
left_delt_move2 = A_move2_filtered_df["BIP10"]
left_delt_move2 *= 1e6

# rectifying and building envelope #
rectified_delt_move2 = left_delt_move2.abs()
enveloped_delt_move2 = sosfiltfilt(sos, x=rectified_delt_move2)

In [78]:
# taking baseline section
A_move2_delt_baseline = enveloped_delt_move2[9*sf:15*sf] # 10 secs for baseline values
p99 = np.percentile(A_move2_delt_baseline, 99)
mu_move2_delt_baseline, sigma_move2_delt_baseline = A_move2_delt_baseline.mean(), A_move2_delt_baseline.std()

In [79]:
# set thresh and get activity
p99_thresh = p99 * 4
std_thresh_move2_delt = mu_move2_delt_baseline + 18 * sigma_move2_delt_baseline
envelope_activity2_delt = enveloped_delt_move2 > std_thresh_move2_delt

In [80]:
# get on and offsets
min_samples = int(1.5 * sf)         # 1500-ms-Grenze

labels, n_lbl = label(envelope_activity2_delt)
valid = np.zeros_like(envelope_activity2_delt, dtype=bool)

for lbl in range(1, n_lbl + 1):
    idx = np.where(labels == lbl)[0]
    if idx.size >= min_samples:
        valid[idx] = True

# On- & Offsets
Onsets_move2_delt  = np.where(np.diff(valid.astype(int)) ==  1)[0] + 1
Offsets_move2_delt = np.where(np.diff(valid.astype(int)) == -1)[0] + 1

In [81]:
new_onsets_move2_delt, new_offsets_move2_delt = take_out_short_off_onset(Onsets_move2_delt, Offsets_move2_delt, 0.2, sf)

In [82]:
plt.plot(A_move2_filtered_df["Sync_Time (s)"], enveloped_delt_move2, "b", label="Envelope")
#plt.plot(A_move1_filtered_df["Sync_Time (s)"][:-1], np.diff(enveloped_move), "green", label="diff envelope")
plt.axhline(std_thresh_move2_delt, color="r", linestyle="--", label="Threshold")
#plt.axhline(diff_p99_thresh, color="r", linestyle="-.", label="thresh for diff")
for x in new_onsets_move2_delt:
    plt.axvline(A_move2_filtered_df["Sync_Time (s)"][x], ls="--", c="g")
for x in new_offsets_move2_delt:
    plt.axvline(A_move2_filtered_df["Sync_Time (s)"][x], ls="--", c="k")
plt.legend()
plt.xlabel("Time (s)")
plt.ylabel("Amplitude (µV)")
plt.title("Muscle Activity Detection for move 2 -> left delt - thresh: mean+18*std + new_on-/offsets")
plt.show()